In [1]:
import requests
import math
import pandas as pd
from requests.exceptions import ReadTimeout, ConnectTimeout

In [2]:
# 读取高德地图用户密钥
def read_key():
    with open('./amap_user_key.txt','r') as f:
        key_str = f.read()
    return key_str

# 结构化地址转换成经纬度
def transform(geo):
    parameters = { "address": geo,  "key":  read_key() }
    base =  "https://restapi.amap.com/v3/geocode/geo"    
    try:
        response = requests.get(base, parameters, timeout=2)
        if response.status_code == 200:
            answer = response.json()
            loc = answer['geocodes'][0].get('location')
        else:
            pass
    except (ReadTimeout, ConnectTimeout):
        # ConnectTimeout指的是建立连接所用的时间，适用于网络状况正常的情况下，
        # 两端连接所用的时间。ReadTimeout指的是建立连接后从服务器读取到可用资源所用的时间。
        pass
    return loc

In [3]:
def search_around(proj_name, radi, search_type='120301|120302'):
    # 高德API Web服务周边搜索功能URL基本字段
    # 默认搜索类型，为
    # 120301：商务住宅，住宅区，别墅
    # 120302：商务住宅，住宅区，住宅小区
    base = "https://restapi.amap.com/v3/place/around"
    # 第一次运行，获得POI的数目
    parameters ={'key' : read_key(),
                 'location' : transform(proj_name),
                 'types' : search_type,
                 'radius' : radi
                 }
    try:
        response = requests.get(base, parameters, timeout=2)
        if response.status_code == 200:
            answer = response.json()
        else:
            pass
    except (ReadTimeout, ConnectTimeout):
        pass
    POIs_num = answer['count']
    # 把所有页的POI全部抓取下来
    pages_num = math.ceil(int(POIs_num) / 20)
    results = pd.DataFrame(columns=['高德ID','高德名称', 'POI类型', '坐标', '距离', '地址'])
    for i in range(1 , pages_num+1):
        parameters.update({ 'page' : i })
        response = requests.get(base, parameters)
        answer = response.json()
        # 将信息存入DataFrame
        for i in answer['pois'][:]:
            info_dict={'高德ID' :  i.get('id'),
                       '高德名称': i.get('name'),
                       'POI类型':  i.get('typecode'),
                       '坐标':     i.get('location'),
                       '距离':     i.get('distance'), 
                       '地址':     i.get('address')}
            results = results.append(info_dict,ignore_index=True)   
    print("{},坐标[{}],周围{}米的小区搜索完成".format(proj_name,parameters['location'],radi))
    return results

In [4]:
s = search_around("上海市奉贤区传悦坊",3000)

上海市奉贤区传悦坊,坐标[121.486477,30.918412],周围3000米的小区搜索完成


In [5]:
s.to_csv('./Amap_info.csv', encoding='utf_8_sig')